```
--- Day 7: Amplification Circuit ---

Based on the navigational maps, you're going to need to send more power to your ship's thrusters to reach Santa in time. To do this, you'll need to configure a series of amplifiers already installed on the ship.

There are five amplifiers connected in series; each one receives an input signal and produces an output signal. They are connected such that the first amplifier's output leads to the second amplifier's input, the second amplifier's output leads to the third amplifier's input, and so on. The first amplifier's input value is 0, and the last amplifier's output leads to your ship's thrusters.

    O-------O  O-------O  O-------O  O-------O  O-------O
0 ->| Amp A |->| Amp B |->| Amp C |->| Amp D |->| Amp E |-> (to thrusters)
    O-------O  O-------O  O-------O  O-------O  O-------O
The Elves have sent you some Amplifier Controller Software (your puzzle input), a program that should run on your existing Intcode computer. Each amplifier will need to run a copy of the program.

When a copy of the program starts running on an amplifier, it will first use an input instruction to ask the amplifier for its current phase setting (an integer from 0 to 4). Each phase setting is used exactly once, but the Elves can't remember which amplifier needs which phase setting.

The program will then call another input instruction to get the amplifier's input signal, compute the correct output signal, and supply it back to the amplifier with an output instruction. (If the amplifier has not yet received an input signal, it waits until one arrives.)

Your job is to find the largest output signal that can be sent to the thrusters by trying every possible combination of phase settings on the amplifiers. Make sure that memory is not shared or reused between copies of the program.

For example, suppose you want to try the phase setting sequence 3,1,2,4,0, which would mean setting amplifier A to phase setting 3, amplifier B to setting 1, C to 2, D to 4, and E to 0. Then, you could determine the output signal that gets sent from amplifier E to the thrusters with the following steps:

Start the copy of the amplifier controller software that will run on amplifier A. At its first input instruction, provide it the amplifier's phase setting, 3. At its second input instruction, provide it the input signal, 0. After some calculations, it will use an output instruction to indicate the amplifier's output signal.
Start the software for amplifier B. Provide it the phase setting (1) and then whatever output signal was produced from amplifier A. It will then produce a new output signal destined for amplifier C.
Start the software for amplifier C, provide the phase setting (2) and the value from amplifier B, then collect its output signal.
Run amplifier D's software, provide the phase setting (4) and input value, and collect its output signal.
Run amplifier E's software, provide the phase setting (0) and input value, and collect its output signal.
The final output signal from amplifier E would be sent to the thrusters. However, this phase setting sequence may not have been the best one; another sequence might have sent a higher signal to the thrusters.

Here are some example programs:

Max thruster signal 43210 (from phase setting sequence 4,3,2,1,0):

3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0
Max thruster signal 54321 (from phase setting sequence 0,1,2,3,4):

3,23,3,24,1002,24,10,24,1002,23,-1,23,
101,5,23,23,1,24,23,23,4,23,99,0,0
Max thruster signal 65210 (from phase setting sequence 1,0,4,3,2):

3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,
1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0
Try every combination of phase settings on the amplifiers. What is the highest signal that can be sent to the thrusters?
```

In [197]:
pos(p) = p + 1;
ref(a,p) = a[pos(a[pos(p)])];
@enum Op plus=1 mul=2 stor=3 out=4 halt=99
@enum Mode posi=0 imm=1

struct Operation 
   op::Op 
   mode1::Mode
   mode2::Mode
   mode3::Mode
end
getOps(str::String) = getOps(parse(Int,str))
function getOps(num)  
    
        op = Op(num % 100)
        mode1 = Mode((num ÷ 100)  % 10)
        mode2 = Mode((num ÷ 1000)  % 10)
        mode3 = Mode((num ÷ 10000)  % 10)
        
        Operation(op,mode1,mode2,mode3)
end

function op1(operation,p,prog) 
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
    
    prog[pos(prog[pos(p+3)])] = p1 + p2
    p + 4
end

function op2(operation,p,prog) 
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end    
    
    prog[pos(prog[pos(p+3)])] = p1 * p2
    p + 4
end

function op3(operation,p,prog) 
#    println("entering stor ","p ",p," prog ",prog, " input ",input)
    p1 = prog[pos(p+1)]
    #println("storage position",p1)
    prog[pos(p1)] = input
    #println("exiting stor")  
    p + 2
end

function op4(operation,p,prog,input) 
    output = []
    #println("entering out")     
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    #println("output:",p1)
    push!(output,p1)
    #println("exiting out $output")    
    (p + 2,output)
end

function execute(op::Operation, p, prog)
    if op.op == plus
        return op1(op,p,prog)
    end
    if op.op == mul
        return op2(op,p,prog)
    end
    if op.op == stor
        return op3(op,p,prog,1)
    end
    if op.op == out
        return op4(op,p,prog)
    end
    error("panic executing unknown op")
end

function execute(prog)
    p = 0
    op = getOps(prog[pos(p)])
    #println("op: ", op)
    while op.op != halt
        #println(prog)
        p = execute(op,p,prog)   
        #println("instruction: ",p, " -> ",prog[pos(p)])
        op = getOps(prog[pos(p)])
    end 
    prog
end;
println(execute([1,9,10,3,2,3,11,0,99,30,40,50]) == [3500,9,10,70,2,3,11,0,99,30,40,50])
println(execute([1,0,0,0,99]) == [2,0,0,0,99])
println(execute([2,3,0,3,99]) == [2,3,0,6,99])
println(execute([2,4,4,5,99,0]) == [2,4,4,5,99,9801])
println(execute([1,1,1,4,99,5,6,0,99]) == [ 30,1,1,4,2,5,6,0,99])

# Opcode 5 is jump-if-true: 
# if the first parameter is non-zero, 
# it sets the instruction pointer to the value from the second parameter. 
# Otherwise, it does nothing.

function op5(operation,p,prog) 
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
    
    if p1 != 0
        return p2
    else
        return p + 3
    end
end

# Opcode 6 is jump-if-false: 
# if the first parameter is zero, 
# it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing.

function op6(operation,p,prog) 
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
    
    if p1 == 0
        return p2
    else
        return p + 3
    end
end

# Opcode 7 is less than: 
# if the first parameter is less than the second parameter, 
# it stores 1 in the position given by the third parameter. Otherwise, it stores 0.

function op7(operation,p,prog) 
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
    
    if p1 < p2
        prog[pos(prog[pos(p+3)])] = 1
    else
        prog[pos(prog[pos(p+3)])] = 0
    end
    p + 4
end

# Opcode 8 is equals: 
# if the first parameter is equal to the second parameter, 
# it stores 1 in the position given by the third parameter. Otherwise, it stores 0.

function op8(operation,p,prog) 
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
    
    if p1 == p2
        prog[pos(prog[pos(p+3)])] = 1
    else
        prog[pos(prog[pos(p+3)])] = 0
    end
    p + 4
end

@enum Op plus=1 mul=2 stor=3 out=4 jnz=5 jez=6 lt=7 eq=8 halt=99
@enum Mode posi=0 imm=1

struct Operation 
   op::Op 
   mode1::Mode
   mode2::Mode
   mode3::Mode
end

function execute(op::Operation, p, prog, input,output)
    if op.op == plus
        return (op1(op,p,prog),input,output)
    end
    if op.op == mul
        return (op2(op,p,prog),input,output)
    end
    if op.op == stor
        #println(length(input))
        next = input[1]
        input=input[2:end]
        #println(next)
        #println(length(input))
        return (op3(op,p,prog,next),input,output)
    end
    if op.op == out
        (p1,output) = op4(op,p,prog,output)
        #println(output)
        return (p1,input,output)
    end
    if op.op == jnz
        return (op5(op,p,prog),input,output)
    end
    if op.op == jez
        return (op6(op,p,prog),input,output)
    end   
    if op.op == lt
        return (op7(op,p,prog),input,output)
    end   
    if op.op == eq
        return (op8(op,p,prog),input,output)
    end    
    
    error("panic executing unknown op")
end

function execute(prog,p,input,output)
    op = getOps(prog[pos(p)])
    #println("op: ", op)
    while op.op != halt
        #println(prog)
        (p,input,output) = execute(op,p,prog,input,output)   
        #println("instruction: ",p, " -> ",prog[pos(p)])
        op = getOps(prog[pos(p)])
    end 
    (prog,p,output)
end

# The above example program uses an input instruction to ask for a single number. 
# The program will then 
# output 999 if the input value is below 8,
# output 1000 if the input value is equal to 8, 
# or output 1001 if the input value is greater than 8.

execute(
    [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
    ,0,[-8],[]);

execute(
    [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
    ,0,[8]);

a = execute(
    [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
    ,0,[16],[]);

#example usage
#using DelimitedFiles
#prog = readdlm("input.txt", '\t', Int, ',')
#execute(prog,5);
a

true
true
true
true
true
output:1000


([3, 21, 1008, 21, 8, 20, 1005, 20, 22, 107  …  1000, 1, 20, 4, 20, 1105, 1, 46, 98, 99], 46, Any[1001])

In [6]:
using DelimitedFiles
get_amp() = readdlm("input.txt", '\t', Int, ',')

get_amp (generic function with 1 method)

In [210]:
# Max thruster signal 43210 (from phase setting sequence 4,3,2,1,0):
function circuit(a,prog)   
    (amp1,p,output) = execute(prog,0,[a[1],0],[]); 
    (amp2,p,output) = execute(prog,0,[a[2],output[1]],[]); 
    (amp3,p,output) = execute(prog,0,[a[3],output[1]],[]); 
    (amp4,p,output) = execute(prog,0,[a[4],output[1]],[]); 
    (amp5,p,output) = execute(prog,0,[a[5],output[1]],[]); 
    output[1]
end
test_amp() = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
circuit([4,3,2,1,0],test_amp())


43210

In [211]:
#65210
test_amp() = [3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0]
circuit([1,0,4,3,2],test_amp())

65210

In [213]:
import Pkg; Pkg.add("Combinatorics")
using Combinatorics
all_inputs = permutations([0,1,2,3,4]) |> collect;


  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [215]:
all_values = []
for x ∈ all_inputs 
    y = circuit(x,test_amp()) 
    push!(all_values,y)
end
maximum(all_values)

65210

In [217]:
# should return 359142
prog = readdlm("input.txt", '\t', Int, ',')
all_values = []
for x ∈ all_inputs 
    y = circuit(x,prog) 
    push!(all_values,y)
end
maximum(all_values)

359142

```
--- Part Two ---

It's no good - in this configuration, the amplifiers can't generate a large enough output signal to produce the thrust you'll need. The Elves quickly talk you through rewiring the amplifiers into a feedback loop:

      O-------O  O-------O  O-------O  O-------O  O-------O
0 -+->| Amp A |->| Amp B |->| Amp C |->| Amp D |->| Amp E |-.
   |  O-------O  O-------O  O-------O  O-------O  O-------O |
   |                                                        |
   '--------------------------------------------------------+
                                                            |
                                                            v
                                                     (to thrusters)
Most of the amplifiers are connected as they were before; amplifier A's output is connected to amplifier B's input, and so on. However, the output from amplifier E is now connected into amplifier A's input. This creates the feedback loop: the signal will be sent through the amplifiers many times.

In feedback loop mode, the amplifiers need totally different phase settings: integers from 5 to 9, again each used exactly once. These settings will cause the Amplifier Controller Software to repeatedly take input and produce output many times before halting. Provide each amplifier its phase setting at its first input instruction; all further input/output instructions are for signals.

Don't restart the Amplifier Controller Software on any amplifier during this process. Each one should continue receiving and sending signals until it halts.

All signals sent or received in this process will be between pairs of amplifiers except the very first signal and the very last signal. To start the process, a 0 signal is sent to amplifier A's input exactly once.

Eventually, the software on the amplifiers will halt after they have processed the final loop. When this happens, the last output signal from amplifier E is sent to the thrusters. Your job is to find the largest output signal that can be sent to the thrusters using the new phase settings and feedback loop arrangement.

Here are some example programs:

Max thruster signal 139629729 (from phase setting sequence 9,8,7,6,5):

3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,
27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5
Max thruster signal 18216 (from phase setting sequence 9,7,8,5,6):

3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,
-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,
53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10
Try every combination of the new phase settings on the amplifier feedback loop. What is the highest signal that can be sent to the thrusters?

Your puzzle answer was 4374895.




In [352]:
pos(p) = p + 1;
ref(a,p) = a[pos(a[pos(p)])];
@enum Op plus=1 mul=2 stor=3 out=4 halt=99
@enum Mode posi=0 imm=1

struct Operation 
   op::Op 
   mode1::Mode
   mode2::Mode
   mode3::Mode
end
getOps(str::String) = getOps(parse(Int,str))
function getOps(num)  
    
        op = Op(num % 100)
        mode1 = Mode((num ÷ 100)  % 10)
        mode2 = Mode((num ÷ 1000)  % 10)
        mode3 = Mode((num ÷ 10000)  % 10)
        
        Operation(op,mode1,mode2,mode3)
end

function op1(operation,p,prog) 
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
    
    prog[pos(prog[pos(p+3)])] = p1 + p2
    p + 4
end

function op2(operation,p,prog) 
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end    
    
    prog[pos(prog[pos(p+3)])] = p1 * p2
    p + 4
end

function op3(operation,p,prog) 
#    println("entering stor ","p ",p," prog ",prog, " input ",input)
    p1 = prog[pos(p+1)]
    #println("storage position",p1)
    prog[pos(p1)] = input
    #println("exiting stor")  
    p + 2
end

function op4(operation,p,prog,input) 
    output = []
    #println("entering out")     
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    #println("output:",p1)
    push!(output,p1)
    #println("exiting out $output")    
    (p + 2,output)
end

function execute(op::Operation, p, prog)
    if op.op == plus
        return op1(op,p,prog)
    end
    if op.op == mul
        return op2(op,p,prog)
    end
    if op.op == stor
        return op3(op,p,prog,1)
    end
    if op.op == out
        return op4(op,p,prog)
    end
    error("panic executing unknown op")
end

function execute(prog)
    p = 0
    op = getOps(prog[pos(p)])
    #println("op: ", op)
    while op.op != halt
        #println(prog)
        p = execute(op,p,prog)   
        #println("instruction: ",p, " -> ",prog[pos(p)])
        op = getOps(prog[pos(p)])
    end 
    prog
end;
println(execute([1,9,10,3,2,3,11,0,99,30,40,50]) == [3500,9,10,70,2,3,11,0,99,30,40,50])
println(execute([1,0,0,0,99]) == [2,0,0,0,99])
println(execute([2,3,0,3,99]) == [2,3,0,6,99])
println(execute([2,4,4,5,99,0]) == [2,4,4,5,99,9801])
println(execute([1,1,1,4,99,5,6,0,99]) == [ 30,1,1,4,2,5,6,0,99])

# Opcode 5 is jump-if-true: 
# if the first parameter is non-zero, 
# it sets the instruction pointer to the value from the second parameter. 
# Otherwise, it does nothing.

function op5(operation,p,prog) 
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
    
    if p1 != 0
        return p2
    else
        return p + 3
    end
end

# Opcode 6 is jump-if-false: 
# if the first parameter is zero, 
# it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing.

function op6(operation,p,prog) 
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
    
    if p1 == 0
        return p2
    else
        return p + 3
    end
end

# Opcode 7 is less than: 
# if the first parameter is less than the second parameter, 
# it stores 1 in the position given by the third parameter. Otherwise, it stores 0.

function op7(operation,p,prog) 
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
    
    if p1 < p2
        prog[pos(prog[pos(p+3)])] = 1
    else
        prog[pos(prog[pos(p+3)])] = 0
    end
    p + 4
end

# Opcode 8 is equals: 
# if the first parameter is equal to the second parameter, 
# it stores 1 in the position given by the third parameter. Otherwise, it stores 0.

function op8(operation,p,prog) 
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
    
    if p1 == p2
        prog[pos(prog[pos(p+3)])] = 1
    else
        prog[pos(prog[pos(p+3)])] = 0
    end
    p + 4
end

@enum Op plus=1 mul=2 stor=3 out=4 jnz=5 jez=6 lt=7 eq=8 halt=99
@enum Mode posi=0 imm=1

struct Operation 
   op::Op 
   mode1::Mode
   mode2::Mode
   mode3::Mode
end

function execute(op::Operation, p, prog, input,output)
    if op.op == plus
        return (op1(op,p,prog),input,output,false)
    end
    if op.op == mul
        return (op2(op,p,prog),input,output,false)
    end
    if op.op == stor
        if length(input) == 0 
            return (p,input,output,true)
        end
        next = input[1]
        input=input[2:end]
        return (op3(op,p,prog,next),input,output,false)
    end
    if op.op == out
        (p1,output) = op4(op,p,prog,output)
        #println(output)
        return (p1,input,output,false)
    end
    if op.op == jnz
        return (op5(op,p,prog),input,output,false)
    end
    if op.op == jez
        return (op6(op,p,prog),input,output,false)
    end   
    if op.op == lt
        return (op7(op,p,prog),input,output,false)
    end   
    if op.op == eq
        return (op8(op,p,prog),input,output,false)
    end    
    
    error("panic executing unknown op")
end

function execute(prog,p,input,output)
    op = getOps(prog[pos(p)])
    while op.op != halt
        (p,input,output,halt) = execute(op,p,prog,input,output)
        if halt
            return (prog,p,output,:pause)
        end
        op = getOps(prog[pos(p)])
    end 
    (prog,p,output,:over)
end

# The above example program uses an input instruction to ask for a single number. 
# The program will then 
# output 999 if the input value is below 8,
# output 1000 if the input value is equal to 8, 
# or output 1001 if the input value is greater than 8.

println(execute(
    [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
    ,0,[-8],[]))

println(execute(
    [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
    ,0,[8],[]))

a = execute(
    [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
    ,0,[16],[]);

#example usage
#using DelimitedFiles
#prog = readdlm("input.txt", '\t', Int, ',')
#execute(prog,5);
a

true
true
true
true
true
([3, 21, 1008, 21, 8, 20, 1005, 20, 22, 107, 8, 21, 20, 1006, 20, 31, 1106, 0, 36, 98, 0, -8, 1002, 21, 125, 20, 4, 20, 1105, 1, 46, 104, 999, 1105, 1, 46, 1101, 1000, 1, 20, 4, 20, 1105, 1, 46, 98, 99], 46, Any[999], :over)
([3, 21, 1008, 21, 8, 20, 1005, 20, 22, 107, 8, 21, 20, 1006, 20, 31, 1106, 0, 36, 98, 1000, 8, 1002, 21, 125, 20, 4, 20, 1105, 1, 46, 104, 999, 1105, 1, 46, 1101, 1000, 1, 20, 4, 20, 1105, 1, 46, 98, 99], 46, Any[1000], :over)


([3, 21, 1008, 21, 8, 20, 1005, 20, 22, 107  …  1000, 1, 20, 4, 20, 1105, 1, 46, 98, 99], 46, Any[1001], :over)

In [353]:
# Max thruster signal 139629729 (from phase setting sequence 9,8,7,6,5):
function feedback(a,prog) 
    state = :start
    p1,p2,p3,p4,p5=0,0,0,0,0
    amp1,amp2,amp3,amp4,amp5=copy(prog),copy(prog),copy(prog),copy(prog),copy(prog)
    output1,output2,output3,output4,output5=[0],[0],[0],[0],[0]
    (amp1,p1,output1,state) = execute(amp1,p1,[a[1]],[]); 
    (amp2,p2,output2,state) = execute(amp2,p2,[a[2]],[]); 
    (amp3,p3,output3,state) = execute(amp3,p3,[a[3]],[]); 
    (amp4,p4,output4,state) = execute(amp4,p4,[a[4]],[]); 
    (amp5,p5,output5,state) = execute(amp5,p5,[a[5]],[]); 
    
    output5=[0]
    while state != :over
        (amp1,p1,output1,state) = execute(amp1,p1,[output5[1]],[]); 
        (amp2,p2,output2,state) = execute(amp2,p2,[output1[1]],[]); 
        (amp3,p3,output3,state) = execute(amp3,p3,[output2[1]],[]); 
        (amp4,p4,output4,state) = execute(amp4,p4,[output3[1]],[]); 
        (amp5,p5,output5,state) = execute(amp5,p5,[output4[1]],[]); 
    end
    output5[1]
end
test_amp() = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]
feedback([9,8,7,6,5],test_amp())

139629729

In [354]:
#Max thruster signal 18216 (from phase setting sequence 9,7,8,5,6):
test_amp() = [3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10]
feedback([9,7,8,5,6],test_amp())

18216

In [355]:
#65210
test_amp() = [3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0]
feedback([1,0,4,3,2],test_amp())

65210

In [356]:
#echo
test_amp() = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
feedback([4,3,2,1,0],test_amp())

43210

In [359]:
# should return 4374895
all_inputs = permutations([5,6,7,8,9]) |> collect;
prog = readdlm("input.txt", '\t', Int, ',')
all_values = []
for x ∈ all_inputs 
    y = feedback(x,prog) 
    push!(all_values,y)
end
maximum(all_values)

4374895